In [1]:
import pickle
import io
import torch
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
import numpy as np
from model import Iris2LayerClassifier

NUM_OF_MODELS = 500

device = "cuda" if torch.cuda.is_available() else "cpu"

In [2]:
iris = load_iris()
X, y = iris.data, iris.target

X_train, X_test, y_train, y_test = train_test_split(
    X, y, random_state=42, test_size=0.33
)
X_train, X_test, y_train, y_test = (
    torch.tensor(X_train, device=device, dtype=torch.float32),
    torch.tensor(X_test, device=device, dtype=torch.float32),
    torch.tensor(y_train, device=device, dtype=torch.long),
    torch.tensor(y_test, device=device, dtype=torch.long),
)

In [3]:
class CPU_Unpickler(pickle.Unpickler):
    def find_class(self, module, name):
        if module == "torch.storage" and name == "_load_from_bytes":
            return lambda b: torch.load(io.BytesIO(b), map_location="cpu")
        else:
            return super().find_class(module, name)


with open("2_layer_real_models.pickle", "rb") as f:
    real_models = CPU_Unpickler(f).load()
with open("2_layer_generated_models.pickle", "rb") as f:
    generated_models = CPU_Unpickler(f).load()

In [9]:
def evaluate_models(state_dicts, X_test, y_test):
    accuracies = []
    for state_dict in state_dicts:
        model = Iris2LayerClassifier().to(device)
        model.load_state_dict(state_dict)
        model.eval()
        with torch.inference_mode():
            y_pred = model(X_test)
            _, predicted_labels = torch.max(y_pred, 1)
            accuracy = accuracy_score(
                y_test.cpu().numpy(), predicted_labels.cpu().numpy()
            )
            accuracies.append(accuracy)
    return np.array(accuracies)


accuracy_real = evaluate_models(real_models, X_test, y_test)
accuracy_generated = evaluate_models(generated_models, X_test, y_test)

accuracy_real *= 100
accuracy_generated *= 100

print(
    "Acc(R): {:0>5.2f} ± {:0>5.2f}".format(
        np.mean(accuracy_real), np.std(accuracy_real)
    )
)
print(
    "Acc(G): {:0>5.2f} ± {:0>5.2f}".format(
        np.mean(accuracy_generated), np.std(accuracy_generated)
    )
)

Acc(R): 95.64 ± 09.96
Acc(G): 83.44 ± 11.23
